In [1]:
print("hi")

hi


In [2]:
import pyspark
import pandas as pd



In [9]:


# create a sample DataFrame
data = {'Name': ['Alice', 'Bob', 'Charlie'],
        'Age': [25, 30, 35],
        'City': ['New York', 'Paris', 'London']}
df = pd.DataFrame(data)

# save the DataFrame to a CSV file
df.to_csv('data.csv', index=False)


In [13]:
df

,Name,Age,City
0,Alice,25,New York
1,Bob,30,Paris
2,Charlie,35,London


In [4]:
from pyspark.sql import SparkSession

In [7]:
spark = SparkSession.builder.appName("Practice").getOrCreate()

In [8]:
spark

In [10]:
df_pyspark = spark.read.csv("data.csv")

In [16]:
#fixing header column name
df_pyspark = spark.read.option('header',"true").csv("data.csv")

In [17]:
df_pyspark.show()

+-------+---+--------+
|   Name|Age|    City|
+-------+---+--------+
|  Alice| 25|New York|
|    Bob| 30|   Paris|
|Charlie| 35|  London|
+-------+---+--------+



In [19]:
df_pyspark.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- City: string (nullable = true)



In [ ]:
# 

# PySpark Dataframe
## Reading The Dataset
## Checking the Datatypes of the Column(Schema)
## Selecting Columns And Indexing
## Check Describe option similar to Pandas
## Adding Columns
## Dropping columns
## Renaming Columns

In [21]:
df_pyspark.select(['Name','City']).show()

+-------+--------+
|   Name|    City|
+-------+--------+
|  Alice|New York|
|    Bob|   Paris|
|Charlie|  London|
+-------+--------+



In [22]:
df_pyspark.dtypes

[('Name', 'string'), ('Age', 'string'), ('City', 'string')]

In [23]:
df_pyspark.describe().show()

+-------+-------+----+------+
|summary|   Name| Age|  City|
+-------+-------+----+------+
|  count|      3|   3|     3|
|   mean|   null|30.0|  null|
| stddev|   null| 5.0|  null|
|    min|  Alice|  25|London|
|    max|Charlie|  35| Paris|
+-------+-------+----+------+



In [24]:

### Adding Columns in data frame
df_pyspark=df_pyspark.withColumn('Age After 2 year',df_pyspark['Age']+2)

In [25]:
df_pyspark.show()

+-------+---+--------+----------------+
|   Name|Age|    City|Age After 2 year|
+-------+---+--------+----------------+
|  Alice| 25|New York|            27.0|
|    Bob| 30|   Paris|            32.0|
|Charlie| 35|  London|            37.0|
+-------+---+--------+----------------+



In [26]:
### Rename the columns
df_pyspark.withColumnRenamed('Name','New Name').show()

+--------+---+--------+----------------+
|New Name|Age|    City|Age After 2 year|
+--------+---+--------+----------------+
|   Alice| 25|New York|            27.0|
|     Bob| 30|   Paris|            32.0|
| Charlie| 35|  London|            37.0|
+--------+---+--------+----------------+



 # Handling missing value

In [28]:
import numpy as np
# Define the column names for the DataFrame
columns = ['employee_id', 'name', 'department', 'title', 'salary']

# Define the data for the DataFrame
data = [[1, 'John', 'HR', 'Manager', 100000],
        [2, 'Alice', 'Engineering', 'Engineer', 90000],
        [3, 'Bob', 'Finance', np.nan, 80000],
        [4, 'Charlie', 'Marketing', 'Coordinator', np.nan]]

# Create the DataFrame
df = pd.DataFrame(data, columns=columns)

# Print the DataFrame
print(df)

   employee_id     name   department        title    salary
0            1     John           HR      Manager  100000.0
1            2    Alice  Engineering     Engineer   90000.0
2            3      Bob      Finance          NaN   80000.0
3            4  Charlie    Marketing  Coordinator       NaN


In [30]:
# save the DataFrame to a CSV file
df.to_csv('employee.csv', index=False)

In [32]:
df_pyspark = spark.read.csv("employee.csv",header=True,inferSchema=True)

In [33]:
df_pyspark.show()

+-----------+-------+-----------+-----------+--------+
|employee_id|   name| department|      title|  salary|
+-----------+-------+-----------+-----------+--------+
|          1|   John|         HR|    Manager|100000.0|
|          2|  Alice|Engineering|   Engineer| 90000.0|
|          3|    Bob|    Finance|       null| 80000.0|
|          4|Charlie|  Marketing|Coordinator|    null|
+-----------+-------+-----------+-----------+--------+



In [34]:
df_pyspark.drop("employee_id").show()

+-------+-----------+-----------+--------+
|   name| department|      title|  salary|
+-------+-----------+-----------+--------+
|   John|         HR|    Manager|100000.0|
|  Alice|Engineering|   Engineer| 90000.0|
|    Bob|    Finance|       null| 80000.0|
|Charlie|  Marketing|Coordinator|    null|
+-------+-----------+-----------+--------+



In [35]:
df_pyspark.na.drop().show()

+-----------+-----+-----------+--------+--------+
|employee_id| name| department|   title|  salary|
+-----------+-----+-----------+--------+--------+
|          1| John|         HR| Manager|100000.0|
|          2|Alice|Engineering|Engineer| 90000.0|
+-----------+-----+-----------+--------+--------+



In [37]:
## any==how drop if there any null in row
## all = drop all rows with all nan in rows
## threshold if 2, if atleast 2 null values then it will be deleted
## subset = remove from specific col

df_pyspark.na.drop(how="any", thresh=2).show()
df_pyspark.na.drop(how="any", subset=["salary"]).show()

+-----------+-------+-----------+-----------+--------+
|employee_id|   name| department|      title|  salary|
+-----------+-------+-----------+-----------+--------+
|          1|   John|         HR|    Manager|100000.0|
|          2|  Alice|Engineering|   Engineer| 90000.0|
|          3|    Bob|    Finance|       null| 80000.0|
|          4|Charlie|  Marketing|Coordinator|    null|
+-----------+-------+-----------+-----------+--------+

+-----------+-----+-----------+--------+--------+
|employee_id| name| department|   title|  salary|
+-----------+-----+-----------+--------+--------+
|          1| John|         HR| Manager|100000.0|
|          2|Alice|Engineering|Engineer| 90000.0|
|          3|  Bob|    Finance|    null| 80000.0|
+-----------+-----+-----------+--------+--------+



In [45]:
## fill missing value

df_pyspark.na.fill(0).show()
df_pyspark.na.fill("MIssing_value", subset=["title"]).show()

+-----------+-------+-----------+-----------+--------+
|employee_id|   name| department|      title|  salary|
+-----------+-------+-----------+-----------+--------+
|          1|   John|         HR|    Manager|100000.0|
|          2|  Alice|Engineering|   Engineer| 90000.0|
|          3|    Bob|    Finance|       null| 80000.0|
|          4|Charlie|  Marketing|Coordinator|     0.0|
+-----------+-------+-----------+-----------+--------+

+-----------+-------+-----------+-------------+--------+
|employee_id|   name| department|        title|  salary|
+-----------+-------+-----------+-------------+--------+
|          1|   John|         HR|      Manager|100000.0|
|          2|  Alice|Engineering|     Engineer| 90000.0|
|          3|    Bob|    Finance|MIssing_value| 80000.0|
|          4|Charlie|  Marketing|  Coordinator|    null|
+-----------+-------+-----------+-------------+--------+



In [46]:
df_pyspark.show()

+-----------+-------+-----------+-----------+--------+
|employee_id|   name| department|      title|  salary|
+-----------+-------+-----------+-----------+--------+
|          1|   John|         HR|    Manager|100000.0|
|          2|  Alice|Engineering|   Engineer| 90000.0|
|          3|    Bob|    Finance|       null| 80000.0|
|          4|Charlie|  Marketing|Coordinator|    null|
+-----------+-------+-----------+-----------+--------+



In [73]:
# imputing mean
# Calculate the mean of each column
from pyspark.sql.functions import mean
means = df_pyspark.select([mean(c) for c in df_pyspark.columns]).first()
# Calculate the mean of the "salary" column
mean_salary = df_pyspark.select(mean('salary')).collect()[0][0]

In [74]:
mean_salary

90000.0

In [76]:
means

Row(avg(employee_id)=2.5, avg(name)=None, avg(department)=None, avg(title)=None, avg(salary)=90000.0)

In [75]:
df_pyspark = df_pyspark.na.fill(mean_salary, subset=["salary"])

In [ ]:
#filter operations

